In [ ]:
import os
import time
import traceback
from datetime import datetime, timedelta
from distutils import util
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
import pickle

In [ ]:
books_df = pd.read_csv("Books.csv")
books_df.head()

C:\Users\bubuw\AppData\Local\Temp\ipykernel_4116\3747861748.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("Books.csv")


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
isbn_lst = books_df["ISBN"].to_list()
isbn_lst[:5]

['0971880107', '0316666343', '0385504209', '0060928336', '0312195516']

In [ ]:
options = EdgeOptions()
service = EdgeService(EdgeChromiumDriverManager().install())
Webdriver = webdriver.Edge
driver = Webdriver(service=service, options=options)
driver.get("https://www.goodreads.com/book/show/238137.Purity_in_Death")

In [ ]:
# Key: ISBN, value: {description, goodreads_rating, genres, series}
with open("books_data.pickle", "rb") as file:
    books_data = pickle.load(file)

In [ ]:
def get_book_metadata(isbn_lst):
    for isbn in isbn_lst:
        if isbn not in books_data:
            search_box = driver.find_element(by = By.CLASS_NAME, value = "HeaderSearch__input")
            time.sleep(1)
            search_box.send_keys(isbn)
            search_box.send_keys(Keys.ENTER)
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,"BookPageTitleSection__title")))
                # Get Book Title
                title = driver.find_element(by = By.CLASS_NAME, value = "BookPageTitleSection__title").text
                # Get Book Description
                try:
                    desc = driver.find_element(by = By.CLASS_NAME, value = "BookPageMetadataSection__description").text
                except:
                    desc = None
                # Get Genres
                try:
                    genres_raw = driver.find_element(by = By.CLASS_NAME, value = "BookPageMetadataSection__genres").text
                except:
                    genres_raw = None
                # Get Rating
                try:
                    gr_rating = driver.find_element(by = By.CLASS_NAME, value = "RatingStatistics__rating").text
                except:
                    gr_rating = None

                books_data[isbn] = {
                    "Description": desc,
                    "GR_Rating": gr_rating,
                    "Genres": genres_raw,
                    "Series": title.split("\n")[0]
                }
            except:
                books_data[isbn] = {
                    "Description": None,
                    "GR_Rating": None,
                    "Genres": None,
                    "Series": None
                }
                driver.get("https://www.goodreads.com/book/show/238137.Purity_in_Death")
                continue

In [ ]:
# get_book_metadata(isbn_lst)

In [ ]:
# with open("books_data.pickle", "wb") as file:
#     pickle.dump(books_data, file)

In [ ]:
# # Key: ISBN, value: {description, goodreads_rating, genres, series}
# with open("books_data.pickle", "rb") as file:
#     books_data = pickle.load(file)

In [ ]:
with open("books_data.pickle", "rb") as file:
    books_data = pickle.load(file)

# Clean the dictionary

for isbn, book_data in books_data.items():
    # Clean the genres
    genres_str = book_data["Genres"]

    if genres_str is None:
        genres_lst = []
        continue

    if "...more" in genres_str:
        genres_str = genres_str.split("...more")[0]

    genres_lst = genres_str.split("\n")[:-1]

    if len(genres_lst) == 1:
        genres_lst = []
    else:
        genres_lst = genres_lst[1:]

    books_data[isbn]["Genres"] = genres_lst

    # Clean the descriptions (remove the \n)
    desc_str = book_data["Description"]
    desc_str = desc_str.replace("\n", " ")
    books_data[isbn]["Description"] = desc_str

    # Clean the series
    series_str = book_data["Series"]
    # Check if part of a series
    if "#" not in series_str:
        series_str = None
    books_data[isbn]["Series"] = series_str

In [ ]:
books_data_df = pd.DataFrame(books_data).T

In [ ]:
books_data_df

,Description,GR_Rating,Genres,Series
0195153448,"Featuring the authors' extensive, clear, and f...",4.08,"[Mythology, Nonfiction, History, Classics, Ref...",None
0002005018,"It is the year 1934, and in a small town in Ca...",3.87,"[Fiction, Canada, Historical Fiction, Literary...",None
0060973129,An outstanding military history that offers a ...,4.04,"[History, World War II, Nonfiction, Military H...",None
0374157065,"The fascinating, true story of the world's dea...",3.91,"[History, Nonfiction, Science, Medical, Medici...",None
0393045218,"Barber, one the world's leading authorities on...",4.15,"[History, Archaeology, Nonfiction, China, Anth...",None
...,...,...,...,...
0373118481,He had returned to save her... Once a handsom...,3.31,"[Romance, Harlequin, Harlequin Presents, Conte...","This Time, Forever #5"
0373122403,"For better, for worse -- for revenge... Rome ...",3.21,"[Harlequin, Harlequin Presents, Romance, Conte...",None
0373119364,"Have you heard the latest? Don't tell anyone, ...",2.80,"[Harlequin, Romance, Harlequin Presents, Conte...",None
0310205719,"A #1 New York Times bestseller, The Purpose Dr...",3.96,"[Christian, Nonfiction, Religion, Self Help, S...",None


In [ ]:
# Combine the dictionary with the dataframe
combined_df = books_df.merge(books_data_df, left_on='ISBN', right_index=True)
combined_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Description,GR_Rating,Genres,Series
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,"Featuring the authors' extensive, clear, and f...",4.08,"[Mythology, Nonfiction, History, Classics, Ref...",None
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"It is the year 1934, and in a small town in Ca...",3.87,"[Fiction, Canada, Historical Fiction, Literary...",None
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,An outstanding military history that offers a ...,4.04,"[History, World War II, Nonfiction, Military H...",None
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,"The fascinating, true story of the world's dea...",3.91,"[History, Nonfiction, Science, Medical, Medici...",None
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,"Barber, one the world's leading authorities on...",4.15,"[History, Archaeology, Nonfiction, China, Anth...",None
...,...,...,...,...,...,...,...,...,...,...,...,...
64385,0373118481,"Their Wedding Day (This Time, Forever) (Harle...",Emma Darcy,1996,Harlequin,http://images.amazon.com/images/P/0373118481.0...,http://images.amazon.com/images/P/0373118481.0...,http://images.amazon.com/images/P/0373118481.0...,He had returned to save her... Once a handsom...,3.31,"[Romance, Harlequin, Harlequin Presents, Conte...","This Time, Forever #5"
64386,0373122403,ROME'S REVENGE (Harlequin Presents),Sara Craven,2002,Harlequin,http://images.amazon.com/images/P/0373122403.0...,http://images.amazon.com/images/P/0373122403.0...,http://images.amazon.com/images/P/0373122403.0...,"For better, for worse -- for revenge... Rome ...",3.21,"[Harlequin, Harlequin Presents, Romance, Conte...",None
64387,0373119364,Scandalous Bride (Scandals) (Harlequin Presen...,Tisha Hamilton,1998,Harlequin,http://images.amazon.com/images/P/0373119364.0...,http://images.amazon.com/images/P/0373119364.0...,http://images.amazon.com/images/P/0373119364.0...,"Have you heard the latest? Don't tell anyone, ...",2.80,"[Harlequin, Romance, Harlequin Presents, Conte...",None
70583,067155302X,Harvest,Tess Gerritsen,1997,Pocket,http://images.amazon.com/images/P/067155302X.0...,http://images.amazon.com/images/P/067155302X.0...,http://images.amazon.com/images/P/067155302X.0...,Tess Gerritsen adini tüm dünyaya duyuran roman...,4.14,"[Thriller, Mystery, Fiction, Crime, Suspense, ...",None


In [ ]:
combined_df.to_csv("Books_v2.csv", index = False)